# Imports

In [38]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy 

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report

# Data

In [2]:
iris = load_iris()

In [3]:
iris.keys()

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names', 'filename'])

In [4]:
X, y = iris['data'], iris['target']

In [5]:
X[:5]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2]])

In [6]:
y[:5]

array([0, 0, 0, 0, 0])

In [7]:
iris['feature_names']

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [8]:
iris['target_names']

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [9]:
np.unique(y)

array([0, 1, 2])

In [10]:
X.shape  # iris data: 150 samples, 4 features 

(150, 4)

In [11]:
y.shape  # iris target: 150 samples 

(150,)

# Train/Test Split

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   test_size=0.2,
                                                   stratify=y,
                                                   random_state=42)

In [15]:
X_train.shape, y_train.shape  # 훈련 셋: 120개 샘플 (테스트 셋: 30개 샘플)

((120, 4), (120,))

In [16]:
X_train[:5]

array([[4.4, 2.9, 1.4, 0.2],
       [4.9, 2.5, 4.5, 1.7],
       [6.8, 2.8, 4.8, 1.4],
       [4.9, 3.1, 1.5, 0.1],
       [5.5, 2.5, 4. , 1.3]])

In [18]:
y_train[:5]

array([0, 2, 1, 0, 1])

# Train

In [20]:
# 특성 스케일링 
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [21]:
X_train_scaled[:5]

array([[-1.72156775, -0.33210111, -1.34572231, -1.32327558],
       [-1.12449223, -1.22765467,  0.41450518,  0.6517626 ],
       [ 1.14439475, -0.5559895 ,  0.58484978,  0.25675496],
       [-1.12449223,  0.11567567, -1.28894078, -1.45494479],
       [-0.40800161, -1.22765467,  0.13059752,  0.12508575]])

In [22]:
X_test_scaled[:5]

array([[-1.72156775, -0.10821272, -1.40250384, -1.32327558],
       [ 0.30848902, -0.10821272,  0.64163131,  0.78343181],
       [-1.12449223, -1.45154306, -0.2668732 , -0.26992188],
       [-1.00507713, -1.67543145, -0.2668732 , -0.26992188],
       [-1.72156775,  0.33956406, -1.40250384, -1.32327558]])

In [23]:
# logistic regression 모델 생성
logi_reg = LogisticRegression()

In [24]:
# 모델 학습 
logi_reg.fit(X_train_scaled, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

# 모델 평가

## 훈련 셋 평가

In [25]:
logi_reg.score(X_train_scaled, y_train)

0.9583333333333334

In [27]:
# 훈련 셋 예측값 
train_pred = logi_reg.predict(X_train_scaled)
train_pred[:5]

array([0, 1, 1, 0, 1])

In [28]:
confusion_matrix(y_train, train_pred)

array([[40,  0,  0],
       [ 0, 38,  2],
       [ 0,  3, 37]])

In [30]:
report = classification_report(y_train, train_pred, target_names=iris['target_names'])
print(report)

              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        40
  versicolor       0.93      0.95      0.94        40
   virginica       0.95      0.93      0.94        40

    accuracy                           0.96       120
   macro avg       0.96      0.96      0.96       120
weighted avg       0.96      0.96      0.96       120



### 예측 확률, 결정 함수, 소프트맥스 

In [35]:
train_pred[:5]  #> train_pred의 shape: (120,)

array([0, 1, 1, 0, 1])

In [34]:
train_pred_pr = logi_reg.predict_proba(X_train_scaled)  #> 예측 확률 shape: (120, 3)
np.around(train_pred_pr[:5], decimals=3)

array([[0.969, 0.031, 0.   ],
       [0.019, 0.513, 0.469],
       [0.002, 0.764, 0.234],
       [0.957, 0.043, 0.   ],
       [0.025, 0.897, 0.078]])

In [37]:
# 결정 함수
decision = logi_reg.decision_function(X_train_scaled) #> shape: (120, 3)
# decision.shape
decision[:5]

array([[ 5.87992851,  2.45018483, -8.33011334],
       [-2.18187437,  1.13608798,  1.04578639],
       [-3.6053981 ,  2.39516734,  1.21023076],
       [ 5.80803392,  2.70389881, -8.51193274],
       [-1.56377633,  2.00406408, -0.44028775]])

In [39]:
softmax = np.exp(decision) / np.sum(np.exp(decision), axis=1, keepdims=True)
# softmax.shape
np.around(softmax[:5], decimals=3)

array([[0.969, 0.031, 0.   ],
       [0.019, 0.513, 0.469],
       [0.002, 0.764, 0.234],
       [0.957, 0.043, 0.   ],
       [0.025, 0.897, 0.078]])

In [43]:
softmax = scipy.special.softmax(decision, axis=1)
# softmax.shape  # (120, 3)
np.around(softmax[:5], decimals=3)

array([[0.969, 0.031, 0.   ],
       [0.019, 0.513, 0.469],
       [0.002, 0.764, 0.234],
       [0.957, 0.043, 0.   ],
       [0.025, 0.897, 0.078]])

In [45]:
# 결정 함수 = 선형 회귀 결과(y = w0 + x1*w1 + .. + x4*w4)
w0 = logi_reg.intercept_
w0 # (3,) = (n_class,)

array([-0.30528152,  1.90861713, -1.60333562])

In [46]:
W = logi_reg.coef_
W  # shape: (3, 4) = (n_class, n_features)

array([[-1.0888546 ,  1.02392865, -1.79886876, -1.68516907],
       [ 0.53614901, -0.36006071, -0.20459098, -0.80836173],
       [ 0.55270559, -0.66386794,  2.00345974,  2.4935308 ]])

In [50]:
deci_fn = w0 + X_train_scaled @ W.T  # (3,) + (120, 4) @ (4, 3) = (120, 3)
deci_fn[:4]

array([[ 5.87992851,  2.45018483, -8.33011334],
       [-2.18187437,  1.13608798,  1.04578639],
       [-3.6053981 ,  2.39516734,  1.21023076],
       [ 5.80803392,  2.70389881, -8.51193274]])

Logistic Regression

* binary-class
    * 선형 회귀: w0, w1
    * sigmoid
    * 확률
    * 예측 
* multi-class
    * 선형 회귀
    * softmax
    * 확률 
    * 예측 